In [17]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import matplotlib 
from PIL import Image
import PIL
import math
from random import random
from random import seed
from random import randint
import time
from numba import jit

In [14]:
@jit(nopython = True)
def Laplacian(A,deltax,deltat):

    h = len(A)
    #CFL condition delta x => sqrt(D) delta t, where D is the number of dimensions            
    P = np.zeros((h,h))
    result = np.zeros((h,h))
    
    for i in range(0, h):
        for j in range(0,h):
            P[i,j] = (-A[(i+2)%h,j] + 16*A[(i+1)%h,j] + 16*A[(i-1)%h,j] -A[(i-2)%h,j] \
                 -A[i,(j+2)%h] + 16*A[i,(j+1)%h]  + 16*A[i,(j-1)%h] -A[i,(j-2)%h] - 60*A[i,j])/(12*pow(deltax,2))

  
    return P 

def damping(t,K_damp):
    Damping = np.zeros(shape=int(t/deltat))
    Damping[0:1000] = 1*K_damp
    return Damping


def update(N,A1,A2,vel1,vel2,omega,eta,deltax,deltat,index,Damping):
    lam = 2*pow(math.pi,2)/pow(omega,2)
    
    F1 = Laplacian(A1,deltax,deltat) - lam*A1*(pow(A1,2)+pow(A2,2)-pow(eta,2)) - Damping[int(index)]*vel1
    F2 = Laplacian(A2,deltax,deltat) - lam*A2*(pow(A1,2)+pow(A2,2)-pow(eta,2)) - Damping[int(index)]*vel2
    
    B1 = A1
    
    A1 = A1 + deltat*(vel1 + 0.5*deltat*F1) 
    A2 = A2 + deltat*(vel2 + 0.5*deltat*F2) 
    
    vel1 = vel1 + deltat/2*(F1 + Laplacian(A1,deltax,deltat) - lam*A1*(pow(A1,2)+pow(A2,2)-pow(eta,2)) - Damping[int(index)]*vel1) 
    vel2 = vel2 + deltat/2*(F2 + Laplacian(A2,deltax,deltat) - lam*A2*(pow(A1,2)+pow(A2,2)-pow(eta,2)) - Damping[int(index)]*vel2)
    return A1,A2,vel1,vel2
    
#change the values accordlingly    
def wolff(N,t,A1,A2,vel,omega,eta,deltax,deltat,K_damp):

    Damping = damping(t,K_damp)


    vel1 = vel
    vel2 = vel


    for i in np.arange(int(t/deltat)):
        if i % 10 == 0:

        
            plt.imsave('258/'+str(int(round((i*deltat),1))).zfill(3)+'.png',np.sqrt(pow(A1,2)+pow(A2,2)),vmin = 0, vmax = eta,cmap = 'Greys')
            plt.imsave('261/'+str(int(round((i*deltat),1))).zfill(3)+'.png',np.arctan2(A2,A1),vmin = -1*math.pi, vmax = math.pi,cmap = 'twilight')
        A1,A2,vel1,vel2 = update(N,A1,A2,vel1,vel2,omega,eta,deltax,deltat,i,Damping)

    return A1,A2

In [29]:
@jit(nopython = True)
def Laplacian(A,deltax,deltat):

    h = len(A)
    #CFL condition delta x => sqrt(D) delta t, where D is the number of dimensions            
    P = np.zeros((h,h))
    result = np.zeros((h,h))
    
    for i in range(0, h):
        for j in range(0,h):
            P[i,j] = (-A[(i+2)%h,j] + 16*A[(i+1)%h,j] + 16*A[(i-1)%h,j] -A[(i-2)%h,j] \
                 -A[i,(j+2)%h] + 16*A[i,(j+1)%h]  + 16*A[i,(j-1)%h] -A[i,(j-2)%h] - 60*A[i,j])/(12*pow(deltax,2))

  
    return P 



def update(N,A1,A2,vel1,vel2,omega,eta,deltax,deltat,a):
    da_dt_a = 1e-3/a 
    lam = 2*pow(math.pi,2)/pow(omega,2)
    
    F1 = Laplacian(A1,deltax,deltat) - lam*A1*(pow(A1,2)+pow(A2,2)-pow(eta,2))#-3*da_dt_a
    F2 = Laplacian(A2,deltax,deltat) - lam*A2*(pow(A1,2)+pow(A2,2)-pow(eta,2)) #-3*da_dt_a
    
    B1 = A1
    
    A1 = A1 + deltat*(vel1 + 0.5*deltat*F1) 
    A2 = A2 + deltat*(vel2 + 0.5*deltat*F2) 
    
    vel1 = vel1 + deltat/2*(F1 + Laplacian(A1,deltax,deltat) - lam*A1*(pow(A1,2)+pow(A2,2)-pow(eta,2)) ) 
    vel2 = vel2 + deltat/2*(F2 + Laplacian(A2,deltax,deltat) - lam*A2*(pow(A1,2)+pow(A2,2)-pow(eta,2)) )
    a = a + 1e-4
    return A1,A2,vel1,vel2
    
#change the values accordlingly    
def wolff(N,t,A1,A2,vel,omega,eta,deltax,deltat,a):



    vel1 = vel
    vel2 = vel


    for i in np.arange(int(t/deltat)):
        if i % 10 == 0:

        
            plt.imsave('259/'+str(int(round((i*deltat),1))).zfill(3)+'.png',np.sqrt(pow(A1,2)+pow(A2,2)),vmin = 0, vmax = eta,cmap = 'Greys')
            plt.imsave('260/'+str(int(round((i*deltat),1))).zfill(3)+'.png',np.arctan2(A2,A1),vmin = -1*math.pi, vmax = math.pi,cmap = 'twilight')
        A1,A2,vel1,vel2 = update(N,A1,A2,vel1,vel2,omega,eta,deltax,deltat,a)

    return A1,A2

In [30]:
start = time.time()

K_damp = 1
n = 8
deltax = 1
deltat = 0.1
eta = 1
N = pow(2,n)
a = 1e-4
t = 0.5*N*deltax/deltat # light crossing time
t2 = t/2 # half time
t5 = t/5
t10 = t/10 # 1/10 of the ligth crossing time

vel = np.zeros((N,N))
mu, sigma = 0, 0.1 # mean and standard deviation
omega = 5

print(t)



A1 = np.random.normal(mu, sigma, size=(N, N))
A2 = np.random.normal(mu,sigma,size=(N,N))
#A1,A2 = wolff(N,t5,A1,A2,vel,omega,eta,deltax,deltat,K_damp)
A1,A2 = wolff(N,t5,A1,A2,vel,omega,eta,deltax,deltat,a)

end = time.time()
print(end - start)

1280.0
34.67870473861694


In [5]:
Laplacian(np.zeros((10,10)),1,0.1)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [31]:
import scipy.integrate as integrate

import scipy.special as special

result = integrate.quad(lambda x: special.jv(2.5,x), 0, 4.5)

result

(1.1178179380783253, 7.866317216380692e-09)

In [ ]:
H_0 = 2.18e-18
omega_r = 9.24e-5
omega_m = 0.315
omega_lambda = 0.685
omega_k = 0
H = H_0 * np.sqrt(omega_r * pow(a,-4) + omega_m * pow(a,-3) + omega_k * pow(a,-2) + omega_lambda)
t = 1/H_0 * integrate.quad(x/(np.sqrt(omega_r * pow(x,-4) + omega_m * pow(x,-3) + omega_k * pow(x,-2) + omega_lambda),0,a))